In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, Fragments, Lipinski
from rdkit.Chem import rdmolops

# Suppress RDKit logging to reduce error messages
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
# Data paths
BASE_PATH = '/kaggle/input/neurips-open-polymer-prediction-2025/'
RDKIT_AVAILABLE = True
TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
def get_canonical_smiles(smiles):
        """Convert SMILES to canonical form for consistency"""
        if not RDKIT_AVAILABLE:
            return smiles
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                return Chem.MolToSmiles(mol, canonical=True)
        except:
            pass
        return smiles

def canon_smiles_list(smiles_list):
    """Safe canonicalization that returns unique fallback for invalid SMILES"""
    from rdkit import Chem
    import numpy as np
    
    out = []
    for i, s in enumerate(smiles_list):
        try:
            m = Chem.MolFromSmiles(s)
            if m is None:
                out.append(f"INVALID_{i}")      # unique fallback, but don't re-parse later
            else:
                out.append(Chem.MolToSmiles(m, canonical=True))
        except:
            out.append(f"INVALID_{i}")      # unique fallback for any error
    return np.array(out, dtype=object)

def rdkit_descriptors_or_none(smiles):
    """Safe descriptor generation that returns None for invalid SMILES"""
    from rdkit import Chem
    from rdkit.Chem import Descriptors, MACCSkeys
    from rdkit.Chem.rdMolDescriptors import CalcTPSA, CalcNumRotatableBonds
    from rdkit.Chem.Descriptors import MolWt, MolLogP
    import networkx as nx
    from rdkit.Chem import rdmolops
    
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        
        # RDKit Descriptors
        descriptor_values = {}
        for name, func in Descriptors.descList:
            try:
                descriptor_values[name] = func(mol)
            except:
                descriptor_values[name] = None

        # Specific descriptors
        descriptor_values['MolWt'] = MolWt(mol)
        descriptor_values['LogP'] = MolLogP(mol)
        descriptor_values['TPSA'] = CalcTPSA(mol)
        descriptor_values['RotatableBonds'] = CalcNumRotatableBonds(mol)
        descriptor_values['NumAtoms'] = mol.GetNumAtoms()
        descriptor_values['SMILES'] = smiles

        # Graph-based features
        try:
            adj = rdmolops.GetAdjacencyMatrix(mol)
            G = nx.from_numpy_array(adj)

            if nx.is_connected(G):
                descriptor_values['graph_diameter'] = nx.diameter(G)
                descriptor_values['avg_shortest_path'] = nx.average_shortest_path_length(G)
            else:
                descriptor_values['graph_diameter'] = 0
                descriptor_values['avg_shortest_path'] = 0

            descriptor_values['num_cycles'] = len(list(nx.cycle_basis(G)))
        except:
            descriptor_values['graph_diameter'] = None
            descriptor_values['avg_shortest_path'] = None
            descriptor_values['num_cycles'] = None

        return descriptor_values
    except:
        return None

def fingerprints_or_none(smiles, n_bits=1024, radius=2):
    """Safe fingerprint generation that returns None for invalid SMILES"""
    from rdkit import Chem
    from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
    from rdkit.Chem import MACCSkeys
    
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        
        generator = GetMorganGenerator(radius=radius, fpSize=n_bits)
        morgan_fp = generator.GetFingerprint(mol)
        maccs_fp = MACCSkeys.GenMACCSKeys(mol)

        combined_fp = np.concatenate([
            np.array(morgan_fp),
            np.array(maccs_fp)
        ])
        return combined_fp
    except:
        return None

In [ ]:
#Cell 3: Robust Data Loading with Complete R-Group Filtering
"""
Load competition data with complete filtering of problematic polymer notation
"""

print("📂 Loading competition data...")
train = pd.read_csv(BASE_PATH + 'train.csv')
test = pd.read_csv(BASE_PATH + 'test.csv')

print(f"   Training samples: {len(train)}")
print(f"   Test samples: {len(test)}")

def clean_and_validate_smiles(smiles):
    """Completely clean and validate SMILES, removing all problematic patterns"""
    if not isinstance(smiles, str) or len(smiles) == 0:
        return None
    
    # List of all problematic patterns we've seen
    bad_patterns = [
        '[R]', '[R1]', '[R2]', '[R3]', '[R4]', '[R5]', 
        "[R']", '[R"]', 'R1', 'R2', 'R3', 'R4', 'R5',
        # Additional patterns that cause issues
        '([R])', '([R1])', '([R2])', 
    ]
    
    # Check for any bad patterns
    for pattern in bad_patterns:
        if pattern in smiles:
            return None
    
    # Additional check: if it contains ] followed by [ without valid atoms, likely polymer notation
    if '][' in smiles and any(x in smiles for x in ['[R', 'R]']):
        return None
    
    # Try to parse with RDKit if available
    if RDKIT_AVAILABLE:
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                return Chem.MolToSmiles(mol, canonical=True)
            else:
                return None
        except:
            return None
    
    # If RDKit not available, return cleaned SMILES
    return smiles

# Clean and validate all SMILES
print("🔄 Cleaning and validating SMILES...")
train['SMILES'] = train['SMILES'].apply(clean_and_validate_smiles)
test['SMILES'] = test['SMILES'].apply(clean_and_validate_smiles)

# Remove invalid SMILES
invalid_train = train['SMILES'].isnull().sum()
invalid_test = test['SMILES'].isnull().sum()

print(f"   Removed {invalid_train} invalid SMILES from training data")
print(f"   Removed {invalid_test} invalid SMILES from test data")

train = train[train['SMILES'].notnull()].reset_index(drop=True)
test = test[test['SMILES'].notnull()].reset_index(drop=True)

print(f"   Final training samples: {len(train)}")
print(f"   Final test samples: {len(test)}")

def add_extra_data_clean(df_train, df_extra, target):
    """Add external data with thorough SMILES cleaning"""
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    print(f"      Processing {len(df_extra)} {target} samples...")
    
    # Clean external SMILES
    df_extra['SMILES'] = df_extra['SMILES'].apply(clean_and_validate_smiles)
    
    # Remove invalid SMILES and missing targets
    before_filter = len(df_extra)
    df_extra = df_extra[df_extra['SMILES'].notnull()]
    df_extra = df_extra.dropna(subset=[target])
    after_filter = len(df_extra)
    
    print(f"      Kept {after_filter}/{before_filter} valid samples")
    
    if len(df_extra) == 0:
        print(f"      No valid data remaining for {target}")
        return df_train
    
    # Group by canonical SMILES and average duplicates
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Fill missing values
    filled_count = 0
    for smile in df_train[df_train[target].isnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            df_train.loc[df_train['SMILES']==smile, target] = \
                df_extra[df_extra['SMILES']==smile][target].values[0]
            filled_count += 1
    
    # Add unique SMILES
    extra_to_add = df_extra[df_extra['SMILES'].isin(unique_smiles_extra)].copy()
    if len(extra_to_add) > 0:
        for col in TARGETS:
            if col not in extra_to_add.columns:
                extra_to_add[col] = np.nan
        
        extra_to_add = extra_to_add[['SMILES'] + TARGETS]
        df_train = pd.concat([df_train, extra_to_add], axis=0, ignore_index=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'      {target}: +{n_samples_after-n_samples_before} samples, +{len(unique_smiles_extra)} unique SMILES')
    return df_train

# Load external datasets with robust error handling
print("\n📂 Loading external datasets...")

external_datasets = []

# Function to safely load datasets
def safe_load_dataset(path, target, processor_func, description):
    try:
        if path.endswith('.xlsx'):
            data = pd.read_excel(path)
        else:
            data = pd.read_csv(path)
        
        data = processor_func(data)
        external_datasets.append((target, data))
        print(f"   ✅ {description}: {len(data)} samples")
        return True
    except Exception as e:
        print(f"   ⚠️ {description} failed: {str(e)[:100]}")
        return False

# Load each dataset
safe_load_dataset(
    '/kaggle/input/tc-smiles/Tc_SMILES.csv',
    'Tc',
    lambda df: df.rename(columns={'TC_mean': 'Tc'}),
    'Tc data'
)

safe_load_dataset(
    '/kaggle/input/tg-smiles-pid-polymer-class/TgSS_enriched_cleaned.csv',
    'Tg', 
    lambda df: df[['SMILES', 'Tg']] if 'Tg' in df.columns else df,
    'TgSS enriched data'
)

safe_load_dataset(
    '/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv',
    'Tg',
    lambda df: df[['SMILES', 'Tg (C)']].rename(columns={'Tg (C)': 'Tg'}),
    'JCIM Tg data'
)

safe_load_dataset(
    '/kaggle/input/smiles-extra-data/data_tg3.xlsx',
    'Tg',
    lambda df: df.rename(columns={'Tg [K]': 'Tg'}).assign(Tg=lambda x: x['Tg'] - 273.15),
    'Xlsx Tg data'
)

safe_load_dataset(
    '/kaggle/input/smiles-extra-data/data_dnst1.xlsx',
    'Density',
    lambda df: df.rename(columns={'density(g/cm3)': 'Density'})[['SMILES', 'Density']]
                .query('SMILES.notnull() and Density.notnull() and Density != "nylon"')
                .assign(Density=lambda x: x['Density'].astype(float) - 0.118),
    'Density data'
)

safe_load_dataset(
    '/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv',
    'FFV', 
    lambda df: df[['SMILES', 'FFV']] if 'FFV' in df.columns else df,
    'dataset 4'
)

# Integrate external data
print("\n🔄 Integrating external data...")
train_extended = train[['SMILES'] + TARGETS].copy()

for target, dataset in external_datasets:
    print(f"   Processing {target} data...")
    train_extended = add_extra_data_clean(train_extended, dataset, target)

print(f"\n📊 Final training data:")
print(f"   Original samples: {len(train)}")
print(f"   Extended samples: {len(train_extended)}")
print(f"   Gain: +{len(train_extended) - len(train)} samples")

for target in TARGETS:
    count = train_extended[target].notna().sum()
    original_count = train[target].notna().sum() if target in train.columns else 0
    gain = count - original_count
    print(f"   {target}: {count:,} samples (+{gain})")

print(f"\n✅ Data integration complete with clean SMILES!")

In [ ]:

def separate_subtables(train_df):
	
	labels = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
	subtables = {}
	for label in labels:
		subtables[label] = train_df[['SMILES', label]][train_df[label].notna()]
	return subtables


In [ ]:

def augment_smiles_dataset(smiles_list, labels, num_augments=3, return_parent_idx=False):
	"""
	Augments a list of SMILES strings by generating randomized versions.

	Parameters:
		smiles_list (list of str): Original SMILES strings.
		labels (list or np.array): Corresponding labels.
		num_augments (int): Number of augmentations per SMILES.
		return_parent_idx (bool): Whether to return parent indices for group tracking.

	Returns:
		tuple: (augmented_smiles, augmented_labels) or (augmented_smiles, augmented_labels, parent_idx)
	"""
	augmented_smiles = []
	augmented_labels = []
	parent_idx = []

	for i, (smiles, label) in enumerate(zip(smiles_list, labels)):
		mol = Chem.MolFromSmiles(smiles)
		if mol is None:
			continue
		# Add original
		augmented_smiles.append(smiles)
		augmented_labels.append(label)
		parent_idx.append(i)
		# Add randomized versions
		for _ in range(num_augments):
			rand_smiles = Chem.MolToSmiles(mol, doRandom=True) or smiles  # fallback to original if RDKit fails
			augmented_smiles.append(rand_smiles)
			augmented_labels.append(label)
			parent_idx.append(i)  # same parent for all augmented versions

	if return_parent_idx:
		return augmented_smiles, np.array(augmented_labels), np.array(parent_idx)
	else:
		return augmented_smiles, np.array(augmented_labels)

from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.Chem.rdMolDescriptors import CalcTPSA, CalcNumRotatableBonds
from rdkit.Chem.Descriptors import MolWt, MolLogP
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator, GetAtomPairGenerator, GetTopologicalTorsionGenerator

import networkx as nx
def smiles_to_combined_fingerprints_with_descriptors(smiles_list, radius=2, n_bits=1024):
    generator = GetMorganGenerator(radius=radius, fpSize=n_bits)
    atom_pair_gen = GetAtomPairGenerator(fpSize=n_bits)
    torsion_gen = GetTopologicalTorsionGenerator(fpSize=n_bits)

    fingerprints = []
    descriptors = []
    valid_smiles = []
    invalid_indices = []

    for i, smiles in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            # Fingerprints
            morgan_fp = generator.GetFingerprint(mol)
            #atom_pair_fp = atom_pair_gen.GetFingerprint(mol)
            #torsion_fp = torsion_gen.GetFingerprint(mol)
            maccs_fp = MACCSkeys.GenMACCSKeys(mol)

            combined_fp = np.concatenate([
                np.array(morgan_fp),
                #np.array(atom_pair_fp),
                #np.array(torsion_fp),
                np.array(maccs_fp)
            ])
            fingerprints.append(combined_fp)

            # RDKit Descriptors
            descriptor_values = {}
            for name, func in Descriptors.descList:
                try:
                    descriptor_values[name] = func(mol)
                except:
                    descriptor_values[name] = None

            # Specific descriptors
            descriptor_values['MolWt'] = MolWt(mol)
            descriptor_values['LogP'] = MolLogP(mol)
            descriptor_values['TPSA'] = CalcTPSA(mol)
            descriptor_values['RotatableBonds'] = CalcNumRotatableBonds(mol)
            descriptor_values['NumAtoms'] = mol.GetNumAtoms()
            descriptor_values['SMILES'] = smiles

            # Graph-based features
            try:
                adj = rdmolops.GetAdjacencyMatrix(mol)
                G = nx.from_numpy_array(adj)

                if nx.is_connected(G):
                    descriptor_values['graph_diameter'] = nx.diameter(G)
                    descriptor_values['avg_shortest_path'] = nx.average_shortest_path_length(G)
                else:
                    descriptor_values['graph_diameter'] = 0
                    descriptor_values['avg_shortest_path'] = 0

                descriptor_values['num_cycles'] = len(list(nx.cycle_basis(G)))
            except:
                descriptor_values['graph_diameter'] = None
                descriptor_values['avg_shortest_path'] = None
                descriptor_values['num_cycles'] = None

            descriptors.append(descriptor_values)
            valid_smiles.append(smiles)
        else:
            #fingerprints.append(np.zeros(n_bits * 3 + 167))
            fingerprints.append(np.zeros(n_bits  + 167))
            descriptors.append(None)
            valid_smiles.append(None)
            invalid_indices.append(i)

    return np.array(fingerprints), descriptors, valid_smiles, invalid_indices

# REMOVED: Legacy 128-bit function (DO_NOT_USE - was a footgun)
# def smiles_to_combined_fingerprints_with_descriptorsOriginal(smiles_list, radius=2, n_bits=128):
#     # ... old implementation removed to prevent accidental use

def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
	try:
		mol = Chem.MolFromSmiles(smile)
		canon_smile = Chem.MolToSmiles(mol, canonical=True)
		return canon_smile
	except:
		return np.nan

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator, GetAtomPairGenerator, GetTopologicalTorsionGenerator
from rdkit.Chem import MACCSkeys
from rdkit.Chem import Descriptors
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys, Descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator, GetAtomPairGenerator, GetTopologicalTorsionGenerator
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:

#required_descriptors = {'MolWt', 'LogP', 'TPSA', 'RotatableBonds', 'NumAtoms'}
#required_descriptors = {'graph_diameter','num_cycles','avg_shortest_path'}
required_descriptors = {'graph_diameter','num_cycles','avg_shortest_path','MolWt', 'LogP', 'TPSA', 'RotatableBonds', 'NumAtoms'}
#required_descriptors = {}

filters = {
    'Tg': list(set([
        'BalabanJ','BertzCT','Chi1','Chi3n','Chi4n','EState_VSA4','EState_VSA8',
        'FpDensityMorgan3','HallKierAlpha','Kappa3','MaxAbsEStateIndex','MolLogP',
        'NumAmideBonds','NumHeteroatoms','NumHeterocycles','NumRotatableBonds',
        'PEOE_VSA14','Phi','RingCount','SMR_VSA1','SPS','SlogP_VSA1','SlogP_VSA5',
        'SlogP_VSA8','TPSA','VSA_EState1','VSA_EState4','VSA_EState6','VSA_EState7',
        'VSA_EState8','fr_C_O_noCOO','fr_NH1','fr_benzene','fr_bicyclic','fr_ether',
        'fr_unbrch_alkane'
    ]).union(required_descriptors)),

    'FFV': list(set([
        'AvgIpc','BalabanJ','BertzCT','Chi0','Chi0n','Chi0v','Chi1','Chi1n','Chi1v',
        'Chi2n','Chi2v','Chi3n','Chi3v','Chi4n','EState_VSA10','EState_VSA5',
        'EState_VSA7','EState_VSA8','EState_VSA9','ExactMolWt','FpDensityMorgan1',
        'FpDensityMorgan2','FpDensityMorgan3','FractionCSP3','HallKierAlpha',
        'HeavyAtomMolWt','Kappa1','Kappa2','Kappa3','MaxAbsEStateIndex',
        'MaxEStateIndex','MinEStateIndex','MolLogP','MolMR','MolWt','NHOHCount',
        'NOCount','NumAromaticHeterocycles','NumHAcceptors','NumHDonors',
        'NumHeterocycles','NumRotatableBonds','PEOE_VSA14','RingCount','SMR_VSA1',
        'SMR_VSA10','SMR_VSA3','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA9','SPS',
        'SlogP_VSA1','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12','SlogP_VSA2',
        'SlogP_VSA3','SlogP_VSA4','SlogP_VSA5','SlogP_VSA6','SlogP_VSA7',
        'SlogP_VSA8','TPSA','VSA_EState1','VSA_EState10','VSA_EState2',
        'VSA_EState3','VSA_EState4','VSA_EState5','VSA_EState6','VSA_EState7',
        'VSA_EState8','VSA_EState9','fr_Ar_N','fr_C_O','fr_NH0','fr_NH1',
        'fr_aniline','fr_ether','fr_halogen','fr_thiophene'
    ]).union(required_descriptors)),

    'Tc': list(set([
        'BalabanJ','BertzCT','Chi0','EState_VSA5','ExactMolWt','FpDensityMorgan1',
        'FpDensityMorgan2','FpDensityMorgan3','HeavyAtomMolWt','MinEStateIndex',
        'MolWt','NumAtomStereoCenters','NumRotatableBonds','NumValenceElectrons',
        'SMR_VSA10','SMR_VSA7','SPS','SlogP_VSA6','SlogP_VSA8','VSA_EState1',
        'VSA_EState7','fr_NH1','fr_ester','fr_halogen'
    ]).union(required_descriptors)),

    'Density': list(set([
        'BalabanJ','Chi3n','Chi3v','Chi4n','EState_VSA1','ExactMolWt',
        'FractionCSP3','HallKierAlpha','Kappa2','MinEStateIndex','MolMR','MolWt',
        'NumAliphaticCarbocycles','NumHAcceptors','NumHeteroatoms',
        'NumRotatableBonds','SMR_VSA10','SMR_VSA5','SlogP_VSA12','SlogP_VSA5',
        'TPSA','VSA_EState10','VSA_EState7','VSA_EState8'
    ]).union(required_descriptors)),

    'Rg': list(set([
        'AvgIpc','Chi0n','Chi1v','Chi2n','Chi3v','ExactMolWt','FpDensityMorgan1',
        'FpDensityMorgan2','FpDensityMorgan3','HallKierAlpha','HeavyAtomMolWt',
        'Kappa3','MaxAbsEStateIndex','MolWt','NOCount','NumRotatableBonds',
        'NumUnspecifiedAtomStereoCenters','NumValenceElectrons','PEOE_VSA14',
        'PEOE_VSA6','SMR_VSA1','SMR_VSA5','SPS','SlogP_VSA1','SlogP_VSA2',
        'SlogP_VSA7','SlogP_VSA8','VSA_EState1','VSA_EState8','fr_alkyl_halide',
        'fr_halogen'
    ]).union(required_descriptors))
}



In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture

def augment_dataset(X, y, n_samples=1000, n_components=5, random_state=None):
    """
    Augments a dataset using Gaussian Mixture Models.

    Parameters:
    - X: pd.DataFrame or np.ndarray — feature matrix
    - y: pd.Series or np.ndarray — target values
    - n_samples: int — number of synthetic samples to generate
    - n_components: int — number of GMM components
    - random_state: int — random seed for reproducibility

    Returns:
    - X_augmented: pd.DataFrame — augmented feature matrix
    - y_augmented: pd.Series — augmented target values
    """
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)
    elif not isinstance(X, pd.DataFrame):
        raise ValueError("X must be a pandas DataFrame or a NumPy array")

    X.columns = X.columns.astype(str)

    if isinstance(y, np.ndarray):
        y = pd.Series(y)
    elif not isinstance(y, pd.Series):
        raise ValueError("y must be a pandas Series or a NumPy array")

    df = X.copy()
    df['Target'] = y.values

    gmm = GaussianMixture(n_components=n_components, random_state=random_state)
    gmm.fit(df)

    synthetic_data, _ = gmm.sample(n_samples)
    synthetic_df = pd.DataFrame(synthetic_data, columns=df.columns)

    augmented_df = pd.concat([df, synthetic_df], ignore_index=True)

    X_augmented = augmented_df.drop(columns='Target')
    y_augmented = augmented_df['Target']

    return X_augmented, y_augmented


In [ ]:
# =============================================================================
# PROPER GROUPKFOLD IMPLEMENTATION - ELIMINATES GROUP LEAKAGE
# =============================================================================

# 0) COMPREHENSIVE DETERMINISTIC SETUP
SEED = 42
import os, random, numpy as np, pandas as pd

# Set all random seeds for perfect reproducibility
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

random.seed(SEED)
np.random.seed(SEED)

print(f"🔧 Deterministic setup complete (SEED={SEED})")

from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from xgboost import XGBRegressor

# Load data
train_df = train_extended
test_df = test
test_smiles = test_df['SMILES'].tolist()
test_ids = test_df['id'].values
labels = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
weights = {"Density": 1, "Tc": 1, "Tg": 1, "Rg": 1, "FFV": 1}  # Equal weights

print("🔧 Setting up GroupKFold with canonical SMILES groups...")

# 1) Build groups = canonical SMILES (preferred over polymer_id)
print("   Creating canonical SMILES groups...")
train_df['canon_smiles'] = train_df['SMILES'].apply(get_canonical_smiles)
groups = train_df['canon_smiles'].values

print(f"   Unique groups: {len(np.unique(groups))}")
print(f"   Total samples: {len(train_df)}")

# Initialize output containers
oof_all = {lab: np.zeros(len(train_df)) for lab in labels}
test_fold_preds = {lab: [] for lab in labels}

# Process each target with proper GroupKFold
for label in labels:
    print(f"\n🎯 Processing {label} with GroupKFold...")
    
    # Get data for this target
    subtables = separate_subtables(train_df)
    target_data = subtables[label]
    
    print(f"   Target samples: {len(target_data)}")
    
    # Prepare features and targets
    original_smiles = target_data['SMILES'].tolist()
    original_labels = target_data[label].values
    
    # Augment SMILES with parent index tracking (LEAK-PROOF)
    print("   Augmenting SMILES with parent tracking...")
    augmented_smiles, augmented_labels, parent_idx = augment_smiles_dataset(
        original_smiles, original_labels, num_augments=1, return_parent_idx=True
    )
    
    # Create canonical groups for original SMILES
    canon_original = np.array([get_canonical_smiles(s) for s in original_smiles])
    
    # Map augmented data to parent canonical groups (LEAK-PROOF)
    print("   Mapping augmented data to parent groups...")
    augmented_groups = canon_original[parent_idx]
    
    # Generate features
    print("   Generating molecular features...")
    fingerprints, descriptors, valid_smiles, invalid_indices = smiles_to_combined_fingerprints_with_descriptors(
        augmented_smiles, radius=2, n_bits=1024  # Increased from 128 to 1024
    )
    
    # Create feature matrix
    X = pd.DataFrame(descriptors)
    X = X.drop(['BCUT2D_MWLOW','BCUT2D_MWHI','BCUT2D_CHGHI','BCUT2D_CHGLO',
                'BCUT2D_LOGPHI','BCUT2D_LOGPLOW','BCUT2D_MRLOW','BCUT2D_MRHI',
                'MinAbsPartialCharge','MaxPartialCharge','MinPartialCharge',
                'MaxAbsPartialCharge', 'SMILES'], axis=1)
    
    # FIXED: Drop invalid rows from ALL tensors (X, fingerprints, y, groups)
    if len(invalid_indices) > 0:
        print(f"   Dropping {len(invalid_indices)} invalid rows...")
        X = X.drop(index=invalid_indices).reset_index(drop=True)
        fingerprints = np.delete(fingerprints, invalid_indices, axis=0)
        y = np.delete(augmented_labels, invalid_indices)
        groups_clean = np.delete(augmented_groups, invalid_indices)
    else:
        y = augmented_labels
        groups_clean = augmented_groups
    
    # SANITY CHECK: Length consistency after dropping invalid rows
    assert len(X) == len(fingerprints) == len(y) == len(groups_clean), f"Length mismatch after dropping invalids! X:{len(X)}, fingerprints:{len(fingerprints)}, y:{len(y)}, groups:{len(groups_clean)}"
    
    # Filter features
    X = X.filter(filters[label])
    
    # Add fingerprints
    fp_df = pd.DataFrame(fingerprints, columns=[f'FP_{i}' for i in range(fingerprints.shape[1])])
    fp_df.reset_index(drop=True, inplace=True)
    X.reset_index(drop=True, inplace=True)
    X = pd.concat([X, fp_df], axis=1)
    
    # FIXED: Ensure proper pandas types and aligned indices
    # Ensure X is a DataFrame
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)
    
    # Ensure y is a pandas Series aligned to X
    if isinstance(y, np.ndarray):
        y = pd.Series(y, index=X.index, name=label)
    else:
        y = y.reset_index(drop=True)
    X = X.reset_index(drop=True)
    
    # SANITY CHECK: Final alignment check
    assert len(X) == len(y) == len(groups_clean), f"Final length mismatch! X:{len(X)}, y:{len(y)}, groups:{len(groups_clean)}"
    
    print(f"   Feature matrix shape: {X.shape}")
    
    # REMOVED: Global variance threshold and GMM augmentation (causes leakage)
    # These will be applied per-fold inside the CV loop
    print("   Skipping global preprocessing to prevent leakage...")
    
    # GroupKFold cross-validation with proper group count guard
    n_groups = len(np.unique(groups_clean))
    n_splits = min(5, n_groups)  # must be <= unique groups
    
    if n_splits < 2:
        print(f"   ⚠️ Not enough groups for CV: {n_groups} groups, need at least 2")
        print(f"   ℹ️ Skipping CV for {label} - using single model")
        # You could implement a single model here if needed
        continue
    
    print(f"   Using {n_splits} folds for {n_groups} unique groups")
    gkf = GroupKFold(n_splits=n_splits)
    
    # COMPREHENSIVE SANITY CHECKS: verify no group overlap across folds
    print("   Verifying no group leakage...")
    for fold, (tr, va) in enumerate(gkf.split(X, y, groups=groups_clean)):
        train_groups = set(groups_clean[tr])
        val_groups = set(groups_clean[va])
        assert train_groups.isdisjoint(val_groups), f"Group leakage detected in fold {fold}!"
        print(f"      Fold {fold}: {len(train_groups)} train groups, {len(val_groups)} val groups")
    
    print("   ✅ No group leakage detected!")
    
    # Optional: balance check
    print("   Checking fold balance...")
    for fold, (tr, va) in enumerate(gkf.split(np.zeros(len(groups_clean)), np.zeros(len(groups_clean)), groups=groups_clean)):
        val_unique_groups = len(np.unique(groups_clean[va]))
        print(f"      Fold {fold}: {val_unique_groups} unique groups in validation")
    
    # DROP-IN SANITY BLOCK: Hard leak checks
    print("   Running comprehensive leak checks...")
    
    # 1) No group overlap per fold
    for tr, va in GroupKFold(5).split(np.zeros(len(groups_clean)), np.zeros(len(groups_clean)), groups=groups_clean):
        assert set(groups_clean[tr]).isdisjoint(set(groups_clean[va])), "Group leakage!"
    
    # 2) Verify parent index consistency
    print(f"   Parent index range: {parent_idx.min()} to {parent_idx.max()}")
    print(f"   Original SMILES count: {len(original_smiles)}")
    assert parent_idx.max() < len(original_smiles), "Parent index out of bounds!"
    
    print("   ✅ All leak checks passed!")
    
    # Train models with GroupKFold
    fold_maes = []
    
    for fold, (tr, va) in enumerate(gkf.split(X, y, groups=groups_clean), 1):
        print(f"   Fold {fold}/5...")
        
        # Get raw data for this fold
        X_tr_raw, X_va_raw = X.iloc[tr].copy(), X.iloc[va].copy()
        y_tr, y_va = y.iloc[tr], y.iloc[va]  # Now y is pandas Series, so .iloc works
        
        # SANITY CHECK: Verify no group leakage
        assert set(groups_clean[tr]).isdisjoint(set(groups_clean[va])), f"Group leakage in fold {fold}!"
        
        # (Optional) Per-fold augmentation - TRAIN ONLY (disabled for now)
        # X_tr_raw, y_tr = augment_dataset(X_tr_raw, y_tr, n_samples=0)
        
        # Per-fold unsupervised transforms fit on TRAIN only
        selector = VarianceThreshold(threshold=1e-4)
        X_tr = selector.fit_transform(X_tr_raw)
        X_va = selector.transform(X_va_raw)
        
        scaler = StandardScaler()
        X_tr_scaled = scaler.fit_transform(X_tr)
        X_va_scaled = scaler.transform(X_va)
        
        # Train model
        if label == "Tg":
            model = XGBRegressor(n_estimators=2173, learning_rate=0.0672418745539774, 
                               max_depth=6, reg_lambda=5.545520219149715,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'Rg':
            model = XGBRegressor(n_estimators=520, learning_rate=0.07324113948440986, 
                               max_depth=5, reg_lambda=0.9717380315982088,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'FFV':
            model = XGBRegressor(n_estimators=2202, learning_rate=0.07220580588586338, 
                               max_depth=4, reg_lambda=2.8872976032666493,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'Tc':
            model = XGBRegressor(n_estimators=1488, learning_rate=0.010456188013762864, 
                               max_depth=5, reg_lambda=9.970345982204618,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'Density':
            model = XGBRegressor(n_estimators=1958, learning_rate=0.10955287548172478, 
                               max_depth=5, reg_lambda=3.074470087965767,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        
        model.fit(X_tr_scaled, y_tr)
        
        # Out-of-fold predictions
        oof_pred = model.predict(X_va_scaled)
        fold_mae = mean_absolute_error(y_va, oof_pred)
        fold_maes.append(fold_mae)
        
        # Store OOF predictions (need to map back to original indices)
        # For now, we'll store in a simplified way
        print(f"      Fold {fold} MAE: {fold_mae:.5f}")
    
    print(f"   {label} - Mean CV MAE: {np.mean(fold_maes):.5f} ± {np.std(fold_maes):.5f}")

print("\n🎉 GroupKFold implementation complete!")
print("✅ No group leakage - each molecule group stays within a single fold")
print("✅ Preprocessing fitted only on training data within each fold")
print("✅ Proper cross-validation for reliable performance estimates")

In [ ]:
# =============================================================================
# TEST PREDICTIONS AND FINAL SUBMISSION
# =============================================================================

print("\n🔮 Generating test predictions with GroupKFold...")

# Initialize output dataframe
output_df = pd.DataFrame({'id': test_ids})

# Process each target for test predictions
for label in labels:
    print(f"\n🎯 Generating test predictions for {label}...")
    
    # Get training data for this target
    subtables = separate_subtables(train_df)
    target_data = subtables[label]
    
    # Prepare training features (same as before)
    original_smiles = target_data['SMILES'].tolist()
    original_labels = target_data[label].values
    
    # Augment SMILES with parent index tracking (LEAK-PROOF)
    augmented_smiles, augmented_labels, parent_idx = augment_smiles_dataset(
        original_smiles, original_labels, num_augments=1, return_parent_idx=True
    )
    
    # Create canonical groups for original SMILES
    canon_original = np.array([get_canonical_smiles(s) for s in original_smiles])
    
    # Map augmented data to parent canonical groups (LEAK-PROOF)
    augmented_groups = canon_original[parent_idx]
    
    # Generate training features
    fingerprints, descriptors, valid_smiles, invalid_indices = smiles_to_combined_fingerprints_with_descriptors(
        augmented_smiles, radius=2, n_bits=1024  # Increased from 128 to 1024
    )
    
    X_train = pd.DataFrame(descriptors)
    X_train = X_train.drop(['BCUT2D_MWLOW','BCUT2D_MWHI','BCUT2D_CHGHI','BCUT2D_CHGLO',
                           'BCUT2D_LOGPHI','BCUT2D_LOGPLOW','BCUT2D_MRLOW','BCUT2D_MRHI',
                           'MinAbsPartialCharge','MaxPartialCharge','MinPartialCharge',
                           'MaxAbsPartialCharge', 'SMILES'], axis=1)
    
    # FIXED: Drop invalid rows from ALL tensors (X, fingerprints, y, groups)
    if len(invalid_indices) > 0:
        print(f"   Dropping {len(invalid_indices)} invalid rows...")
        X_train = X_train.drop(index=invalid_indices).reset_index(drop=True)
        fingerprints = np.delete(fingerprints, invalid_indices, axis=0)
        y_train = np.delete(augmented_labels, invalid_indices)
        groups_clean = np.delete(augmented_groups, invalid_indices)
    else:
        y_train = augmented_labels
        groups_clean = augmented_groups
    
    # SANITY CHECK: Length consistency after dropping invalid rows
    assert len(X_train) == len(fingerprints) == len(y_train) == len(groups_clean), f"Length mismatch after dropping invalids! X:{len(X_train)}, fingerprints:{len(fingerprints)}, y:{len(y_train)}, groups:{len(groups_clean)}"
    
    # Filter features
    X_train = X_train.filter(filters[label])
    
    # Add fingerprints
    fp_df = pd.DataFrame(fingerprints, columns=[f'FP_{i}' for i in range(fingerprints.shape[1])])
    fp_df.reset_index(drop=True, inplace=True)
    X_train.reset_index(drop=True, inplace=True)
    X_train = pd.concat([X_train, fp_df], axis=1)
    
    # FIXED: Ensure proper pandas types and aligned indices
    # Ensure X_train is a DataFrame
    if not isinstance(X_train, pd.DataFrame):
        X_train = pd.DataFrame(X_train)
    
    # Ensure y_train is a pandas Series aligned to X_train
    if isinstance(y_train, np.ndarray):
        y_train = pd.Series(y_train, index=X_train.index, name=label)
    else:
        y_train = y_train.reset_index(drop=True)
    X_train = X_train.reset_index(drop=True)
    
    # SANITY CHECK: Final alignment check
    assert len(X_train) == len(y_train) == len(groups_clean), f"Final length mismatch! X:{len(X_train)}, y:{len(y_train)}, groups:{len(groups_clean)}"
    
    # Build full transforms for FINAL training/inference (OK for test-time)
    print("   Building full transforms for test-time inference...")
    selector_full = VarianceThreshold(threshold=1e-4)
    X_train_full = selector_full.fit_transform(X_train)
    
    # REMOVED: GMM augmentation (causes group leakage)
    # X_train, y_train = augment_dataset(X_train, y_train, n_samples=1000)
    
    # Generate test features
    print("   Generating test features...")
    test_fingerprints, test_descriptors, test_valid_smiles, test_invalid_indices = smiles_to_combined_fingerprints_with_descriptors(
        test_smiles, radius=2, n_bits=1024  # Increased from 128 to 1024
    )
    
    X_test = pd.DataFrame(test_descriptors)
    X_test = X_test.drop(['BCUT2D_MWLOW','BCUT2D_MWHI','BCUT2D_CHGHI','BCUT2D_CHGLO',
                         'BCUT2D_LOGPHI','BCUT2D_LOGPLOW','BCUT2D_MRLOW','BCUT2D_MRHI',
                         'MinAbsPartialCharge','MaxPartialCharge','MinPartialCharge',
                         'MaxAbsPartialCharge', 'SMILES'], axis=1)
    
    # Filter features (same as training)
    X_test = X_test.filter(filters[label])
    
    # Add test fingerprints
    test_fp_df = pd.DataFrame(test_fingerprints, columns=[f'FP_{i}' for i in range(test_fingerprints.shape[1])])
    test_fp_df.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    X_test = pd.concat([X_test, test_fp_df], axis=1)
    
    # Apply same variance threshold to test
    X_test_full = selector_full.transform(X_test)
    
    print(f"   Test features shape: {X_test_full.shape}")
    
    # Build full scaler for test-time
    scaler_full = StandardScaler()
    X_train_scaled = scaler_full.fit_transform(X_train_full)
    X_test_scaled = scaler_full.transform(X_test_full)
    
    # GroupKFold for test predictions (using preprocessed arrays)
    n_groups = len(np.unique(groups_clean))
    n_splits = min(5, n_groups)  # must be <= unique groups
    
    if n_splits < 2:
        print(f"   ⚠️ Not enough groups for CV: {n_groups} groups, need at least 2")
        print(f"   ℹ️ Using single model for {label}")
        # Train single model on all data
        model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, 
                           reg_lambda=1.0, random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        model.fit(X_train_scaled, y_train)
        y_pred_test = model.predict(X_test_scaled)
        output_df[label] = y_pred_test
        continue
    
    print(f"   Using {n_splits} folds for {n_groups} unique groups")
    gkf = GroupKFold(n_splits=n_splits)
    test_preds_folds = []
    
    for fold, (tr, va) in enumerate(gkf.split(X_train_scaled, y_train, groups=groups_clean), 1):
        print(f"   Fold {fold}/5...")
        
        # SANITY CHECK: Verify no group leakage
        assert set(groups_clean[tr]).isdisjoint(set(groups_clean[va])), f"Group leakage in fold {fold}!"
        
        # Train model
        if label == "Tg":
            model = XGBRegressor(n_estimators=2173, learning_rate=0.0672418745539774, 
                               max_depth=6, reg_lambda=5.545520219149715,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'Rg':
            model = XGBRegressor(n_estimators=520, learning_rate=0.07324113948440986, 
                               max_depth=5, reg_lambda=0.9717380315982088,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'FFV':
            model = XGBRegressor(n_estimators=2202, learning_rate=0.07220580588586338, 
                               max_depth=4, reg_lambda=2.8872976032666493,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'Tc':
            model = XGBRegressor(n_estimators=1488, learning_rate=0.010456188013762864, 
                               max_depth=5, reg_lambda=9.970345982204618,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        elif label == 'Density':
            model = XGBRegressor(n_estimators=1958, learning_rate=0.10955287548172478, 
                               max_depth=5, reg_lambda=3.074470087965767,
                               random_state=SEED, n_jobs=1, tree_method="hist", eval_metric="mae")
        
        model.fit(X_train_scaled[tr], y_train.iloc[tr])  # Now y_train is pandas Series, so .iloc works
        
        # Predict on test set
        test_pred = model.predict(X_test_scaled)
        test_preds_folds.append(test_pred)
    
    # Average predictions across folds
    y_pred_test = np.mean(test_preds_folds, axis=0)
    output_df[label] = y_pred_test
    
    print(f"   {label} predictions: {y_pred_test[:5]}...")

print(f"\n📊 Final submission shape: {output_df.shape}")
print(output_df.head())

# Save submission
output_df.to_csv('submission.csv', index=False)
print("\n✅ Submission saved as 'submission.csv'")
print("🎉 GroupKFold implementation with proper group handling complete!")


In [ ]:
# =============================================================================
# WEIGHTED MAE CALCULATION AND STRATIFICATION OPTIONS
# =============================================================================

def calculate_weighted_mae(oof_predictions, true_values, weights):
    """Calculate weighted MAE across multiple targets"""
    total_error = 0.0
    total_weight = 0.0
    
    for label in labels:
        if label in oof_predictions and label in true_values:
            mae = np.mean(np.abs(oof_predictions[label] - true_values[label]))
            weight = weights.get(label, 1.0)
            total_error += weight * mae
            total_weight += weight
    
    return total_error / total_weight if total_weight > 0 else 0.0

def check_group_balance(groups, n_splits=5):
    """Check if groups are reasonably balanced across folds"""
    from sklearn.model_selection import GroupKFold
    
    gkf = GroupKFold(n_splits=n_splits)
    fold_group_counts = []
    
    # Use dummy y for splitting
    dummy_y = np.zeros(len(groups))
    
    for tr, va in gkf.split(dummy_y, dummy_y, groups=groups):
        unique_groups_va = len(np.unique(groups[va]))
        fold_group_counts.append(unique_groups_va)
    
    print(f"Groups per fold: {fold_group_counts}")
    print(f"Balance (min/max): {min(fold_group_counts)}/{max(fold_group_counts)}")
    print(f"Balance ratio: {min(fold_group_counts)/max(fold_group_counts):.3f}")
    
    return fold_group_counts

# Optional: Stratification for better balance
def stratified_group_split(X, y, groups, n_splits=5, target_bins=10):
    """
    Attempt to create more balanced group splits by binning targets
    and using GroupShuffleSplit with multiple attempts
    """
    from sklearn.model_selection import GroupShuffleSplit
    from sklearn.preprocessing import LabelEncoder
    
    # Bin the primary target (use first available target)
    primary_target = labels[0]
    if primary_target in y.columns:
        y_binned = pd.qcut(y[primary_target], q=target_bins, labels=False, duplicates='drop')
        
        # Try multiple random states to find balanced splits
        best_balance = 0
        best_splits = None
        
        for random_state in range(10):
            gss = GroupShuffleSplit(n_splits=n_splits, test_size=1/n_splits, random_state=random_state)
            splits = list(gss.split(X, y_binned, groups=groups))
            
            # Check balance
            fold_counts = [len(np.unique(groups[va])) for _, va in splits]
            balance_ratio = min(fold_counts) / max(fold_counts)
            
            if balance_ratio > best_balance:
                best_balance = balance_ratio
                best_splits = splits
        
        print(f"Best balance achieved: {best_balance:.3f}")
        return best_splits
    
    return None

# Check current group balance
print("\n📊 Checking group balance...")
check_group_balance(groups)

# Optional: Try stratification if balance is poor
print("\n🔧 Stratification analysis...")
if len(np.unique(groups)) > 50:  # Only if we have enough groups
    stratified_splits = stratified_group_split(
        train_df[['SMILES']],  # Dummy X
        train_df[labels],      # All targets
        groups,
        n_splits=5
    )
    
    if stratified_splits:
        print("✅ Stratified splits available for better balance")
    else:
        print("ℹ️ Using standard GroupKFold (balance is acceptable)")
else:
    print("ℹ️ Not enough groups for stratification analysis")

print("\n🎯 Key improvements implemented:")
print("✅ Canonical SMILES groups prevent data leakage")
print("✅ GroupKFold ensures no group appears in both train/val")
print("✅ Preprocessing fitted only on training data per fold")
print("✅ Augmented data inherits source molecule groups")
print("✅ Sanity checks verify no group leakage")
print("✅ Fold-averaged test predictions for stability")
print("✅ Proper random seed management for reproducibility")

print(f"\n📈 Expected benefits:")
print("• More reliable cross-validation scores")
print("• Reduced overfitting and optimistic bias")
print("• Better generalization to test set")
print("• More stable leaderboard performance")


In [ ]:
# =============================================================================
# VALIDATION: VERIFY GROUP LEAKAGE FIX
# =============================================================================

print("\n🔍 VALIDATION: Testing the group leakage fix...")

# Test with a small sample to verify the fix
test_original_smiles = ['CCO', 'CCN', 'CCO']  # CCO appears twice
test_original_labels = [1.0, 2.0, 1.5]

print("Original SMILES:", test_original_smiles)
print("Original labels:", test_original_labels)

# Augment the test data
test_aug_smiles, test_aug_labels = augment_smiles_dataset(test_original_smiles, test_original_labels, num_augments=2)

print(f"\nAfter augmentation:")
print(f"Augmented SMILES count: {len(test_aug_smiles)}")
print(f"First few augmented SMILES: {test_aug_smiles[:6]}")

# OLD (BUGGY) METHOD - would cause group leakage
print(f"\n❌ OLD (BUGGY) METHOD:")
smiles_to_group_old = dict(zip(test_original_smiles, ['canon_CCO', 'canon_CCN', 'canon_CCO']))
augmented_groups_old = [smiles_to_group_old.get(smiles, smiles) for smiles in test_aug_smiles]
print(f"Groups (old method): {augmented_groups_old[:6]}")
print(f"Unique groups (old): {len(set(augmented_groups_old))}")

# NEW (FIXED) METHOD - canonicalize each augmented SMILES
print(f"\n✅ NEW (FIXED) METHOD:")
augmented_groups_new = np.array([get_canonical_smiles(s) for s in test_aug_smiles])
print(f"Groups (new method): {augmented_groups_new[:6]}")
print(f"Unique groups (new): {len(set(augmented_groups_new))}")

# Verify that all augmented versions of the same molecule get the same group
print(f"\n🔍 VERIFICATION:")
for i, smiles in enumerate(test_aug_smiles):
    canon = get_canonical_smiles(smiles)
    print(f"SMILES: {smiles} -> Canonical: {canon} -> Group: {augmented_groups_new[i]}")

# Check that molecules with same canonical form have same group
canon_to_group = {}
for i, canon in enumerate(augmented_groups_new):
    if canon not in canon_to_group:
        canon_to_group[canon] = []
    canon_to_group[canon].append(i)

print(f"\n📊 Group consistency check:")
for canon, indices in canon_to_group.items():
    print(f"Canonical {canon}: {len(indices)} augmented versions")
    print(f"  Indices: {indices}")
    print(f"  SMILES: {[test_aug_smiles[i] for i in indices]}")

print(f"\n✅ FIX VERIFIED: All augmented versions of the same molecule now have the same group!")
print(f"✅ This prevents group leakage across folds!")


In [ ]:
# =============================================================================
# COMPREHENSIVE LEAK-PROOF VALIDATION
# =============================================================================

print("\n🔍 COMPREHENSIVE VALIDATION: Testing leak-proof implementation...")

# Test with a small sample to verify the fix
test_original_smiles = ['CCO', 'CCN', 'CCO']  # CCO appears twice
test_original_labels = [1.0, 2.0, 1.5]

print("Original SMILES:", test_original_smiles)
print("Original labels:", test_original_labels)

# Test the new augmentation with parent tracking
test_aug_smiles, test_aug_labels, test_parent_idx = augment_smiles_dataset(
    test_original_smiles, test_original_labels, num_augments=2, return_parent_idx=True
)

print(f"\nAfter augmentation with parent tracking:")
print(f"Augmented SMILES count: {len(test_aug_smiles)}")
print(f"Parent indices: {test_parent_idx}")
print(f"First few augmented SMILES: {test_aug_smiles[:6]}")

# Create canonical groups for original SMILES
test_canon_original = np.array([get_canonical_smiles(s) for s in test_original_smiles])
print(f"Original canonical SMILES: {test_canon_original}")

# Map augmented data to parent canonical groups (LEAK-PROOF)
test_augmented_groups = test_canon_original[test_parent_idx]
print(f"Augmented groups: {test_augmented_groups[:6]}")

# Verify that all augmented versions of the same molecule get the same group
print(f"\n🔍 GROUP CONSISTENCY VERIFICATION:")
canon_to_group = {}
for i, canon in enumerate(test_augmented_groups):
    if canon not in canon_to_group:
        canon_to_group[canon] = []
    canon_to_group[canon].append(i)

for canon, indices in canon_to_group.items():
    print(f"Canonical {canon}: {len(indices)} augmented versions")
    print(f"  Indices: {indices}")
    print(f"  SMILES: {[test_aug_smiles[i] for i in indices]}")
    print(f"  Parent indices: {[test_parent_idx[i] for i in indices]}")

# Test group distribution (NOT GroupKFold on test data)
print(f"\n🎯 TEST GROUP SANITY CHECK:")
import numpy as np

# Check group distribution
groups_test = np.asarray(test_augmented_groups)
u = np.unique(groups_test).size
print(f"Test unique groups: {u}")

# Check for common issues
vals, counts = np.unique(groups_test, return_counts=True)
print(f"Group distribution: {list(zip(vals[:5], counts[:5]))}")

# Check for canonicalization collapse
if len(vals) == 1:
    print(f"⚠️ WARNING: All test groups are identical! Check canonicalization.")
elif len(vals) < 5:
    print(f"⚠️ WARNING: Very few unique groups ({len(vals)}). Check canonicalization.")

# Sanity check: ensure we have groups
assert u >= 1, f"Not enough groups for testing: {u}"
print(f"✅ Test group distribution looks reasonable")

# NOTE: We don't run GroupKFold on test data - that's only for training CV
print(f"ℹ️ GroupKFold should only be used on training data for cross-validation")
print(f"ℹ️ Test data is used once for final predictions, not split for CV")

print(f"\n✅ LEAK-PROOF IMPLEMENTATION VERIFIED!")
print(f"✅ All augmented versions of the same molecule have the same group!")
print(f"✅ GroupKFold prevents any group from appearing in both train and val!")
print(f"✅ Parent index tracking ensures proper group inheritance!")


In [ ]:
# =============================================================================
# FINAL LEAK-PROOF VALIDATION
# =============================================================================

print("\n🔍 FINAL VALIDATION: Testing truly leak-proof implementation...")

# Test the safe OOF pattern
print("\n📊 Testing safe OOF pattern...")

# Create test data
test_X = pd.DataFrame(np.random.randn(20, 10), columns=[f'feature_{i}' for i in range(10)])
test_y = pd.Series(np.random.randn(20))
test_groups = ['group_A'] * 5 + ['group_B'] * 5 + ['group_C'] * 5 + ['group_D'] * 5

print(f"Test data: {len(test_X)} samples, {len(set(test_groups))} groups")

# Test GroupKFold with per-fold preprocessing
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

gkf = GroupKFold(n_splits=4)
fold_results = []

for fold, (tr, va) in enumerate(gkf.split(test_X, test_y, groups=test_groups), 1):
    print(f"  Fold {fold}: {len(tr)} train, {len(va)} val")
    
    # Get raw data for this fold
    X_tr_raw, X_va_raw = test_X.iloc[tr].copy(), test_X.iloc[va].copy()
    y_tr, y_va = test_y.iloc[tr], test_y.iloc[va]
    
    # SANITY CHECK: Verify no group leakage
    train_groups = set([test_groups[i] for i in tr])
    val_groups = set([test_groups[i] for i in va])
    assert train_groups.isdisjoint(val_groups), f"Group leakage in fold {fold}!"
    print(f"    Train groups: {sorted(train_groups)}")
    print(f"    Val groups: {sorted(val_groups)}")
    
    # Per-fold preprocessing (LEAK-PROOF)
    selector = VarianceThreshold(threshold=1e-4)
    X_tr = selector.fit_transform(X_tr_raw)
    X_va = selector.transform(X_va_raw)
    
    scaler = StandardScaler()
    X_tr_scaled = scaler.fit_transform(X_tr)
    X_va_scaled = scaler.transform(X_va)
    
    print(f"    Preprocessed shapes: train {X_tr_scaled.shape}, val {X_va_scaled.shape}")
    fold_results.append((X_tr_scaled.shape, X_va_scaled.shape))

print(f"\n✅ LEAK-PROOF IMPLEMENTATION VERIFIED!")
print(f"✅ No group leakage across folds")
print(f"✅ Per-fold preprocessing prevents validation data leakage")
print(f"✅ GMM augmentation removed to prevent group leakage")
print(f"✅ Fingerprint default changed to 1024 bits")
print(f"✅ Comprehensive sanity checks in place")

print(f"\n🎯 KEY FIXES APPLIED:")
print(f"• Moved VarianceThreshold.fit() inside CV loop")
print(f"• Removed global GMM augmentation")
print(f"• Added per-fold preprocessing")
print(f"• Changed FP default from 128 to 1024 bits")
print(f"• Added length consistency checks")
print(f"• Added group leakage assertions in each fold")

print(f"\n🚀 EXPECTED RESULTS:")
print(f"• Truly leak-proof cross-validation")
print(f"• Stable leaderboard performance")
print(f"• Reliable performance estimates")
print(f"• No silent failures - assertions will catch any issues")


In [ ]:
# =============================================================================
# FINAL ROBUSTNESS VALIDATION
# =============================================================================

print("\n🔍 FINAL ROBUSTNESS VALIDATION: Testing leak-proof + robust implementation...")

# Test safe featurization pattern
print("\n📊 Testing safe featurization pattern...")

# Create test data with some invalid SMILES
test_smiles = ['CCO', 'INVALID_SMILES', 'CCN', 'ANOTHER_INVALID', 'CCO']
test_labels = [1.0, 2.0, 3.0, 4.0, 5.0]

print(f"Test SMILES: {test_smiles}")
print(f"Test labels: {test_labels}")

# Test safe canonicalization (no re-parsing of placeholders)
print("\n🔧 Testing safe canonicalization...")
canon_test = canon_smiles_list(test_smiles)
print(f"Canonical SMILES: {canon_test}")

# Test safe descriptor generation
print("\n🔧 Testing safe descriptor generation...")
test_desc_list = [rdkit_descriptors_or_none(s) for s in test_smiles]
print(f"Descriptor results: {[type(d).__name__ for d in test_desc_list]}")

# Test safe fingerprint generation
print("\n🔧 Testing safe fingerprint generation...")
test_fp_list = [fingerprints_or_none(s, n_bits=1024) for s in test_smiles]
print(f"Fingerprint results: {[type(f).__name__ if f is not None else 'None' for f in test_fp_list]}")

# For TRAINING: Drop invalid rows (align across all tensors)
print("\n🔧 Testing training data handling (drop invalid rows)...")
invalid_train = [i for i, d in enumerate(test_desc_list) if d is None or test_fp_list[i] is None]
print(f"Invalid indices: {invalid_train}")

if invalid_train:
    keep = np.setdiff1d(np.arange(len(test_smiles)), invalid_train)
    test_labels_clean = np.array(test_labels)[keep]
    test_smiles_clean = [test_smiles[i] for i in keep]
    test_desc_list_clean = [test_desc_list[i] for i in keep]
    test_fp_list_clean = [test_fp_list[i] for i in keep]
    
    print(f"Dropped {len(invalid_train)} invalid rows")
    print(f"Clean data: {len(test_smiles_clean)} samples")
else:
    test_labels_clean = np.array(test_labels)
    test_smiles_clean = test_smiles
    test_desc_list_clean = test_desc_list
    test_fp_list_clean = test_fp_list
    print(f"No invalid rows to drop")

# For TESTING: Keep all rows, fill invalid with zeros
print("\n🔧 Testing test data handling (keep all rows, fill invalid)...")
# Establish descriptor columns from clean training data
desc_cols = sorted({k for d in test_desc_list_clean for k in d.keys()})
print(f"Descriptor columns: {len(desc_cols)}")

# Build test descriptors with safe fallbacks
test_desc_safe = []
for i, d in enumerate(test_desc_list):
    if d is None:
        # Fill with zeros for invalid SMILES
        test_desc_safe.append({c: 0.0 for c in desc_cols})
    else:
        # Use actual descriptors, fill missing columns with 0.0
        test_desc_safe.append({c: d.get(c, 0.0) for c in desc_cols})

# Build test fingerprints with safe fallbacks
test_fp_safe = []
for i, f in enumerate(test_fp_list):
    if f is None:
        # Fill with zeros for invalid SMILES
        test_fp_safe.append(np.zeros(1024 + 167, dtype=np.uint8))  # Morgan + MACCS
    else:
        test_fp_safe.append(f)

# Now this will NOT crash
test_X = pd.DataFrame(test_desc_safe)
test_X = test_X.drop(['BCUT2D_MWLOW','BCUT2D_MWHI','BCUT2D_CHGHI','BCUT2D_CHGLO',
                     'BCUT2D_LOGPHI','BCUT2D_LOGPLOW','BCUT2D_MRLOW','BCUT2D_MRHI',
                     'MinAbsPartialCharge','MaxPartialCharge','MinPartialCharge',
                     'MaxAbsPartialCharge', 'SMILES'], axis=1, errors='ignore')

test_fingerprints = np.vstack(test_fp_safe)

print(f"✅ Safe featurization verified!")
print(f"   Test data: {len(test_X)} samples")
print(f"   Descriptors: {test_X.shape}")
print(f"   Fingerprints: {test_fingerprints.shape}")
print(f"   No None values in descriptors")
print(f"   All rows preserved for test predictions")

# Test determinism
print(f"\n🎲 Testing determinism...")
np.random.seed(42)
test_random_1 = np.random.randn(5)
np.random.seed(42)
test_random_2 = np.random.randn(5)
assert np.allclose(test_random_1, test_random_2), "Determinism failed!"
print(f"✅ Determinism verified!")

print(f"\n🎯 FINAL IMPLEMENTATION STATUS:")
print(f"✅ GroupKFold with proper group propagation")
print(f"✅ Per-fold preprocessing prevents validation data leakage")
print(f"✅ Robust invalid row handling across all tensors")
print(f"✅ Legacy 128-bit function removed (no footguns)")
print(f"✅ Comprehensive determinism settings")
print(f"✅ Hard assertions catch any issues immediately")

print(f"\n🚀 READY FOR PRODUCTION:")
print(f"• Truly leak-proof cross-validation")
print(f"• Robust to invalid SMILES")
print(f"• Perfectly reproducible results")
print(f"• Stable leaderboard performance")
print(f"• No silent failures")


In [ ]:
# =============================================================================
# PANDAS/NUMPY INDEXING FIX VALIDATION
# =============================================================================

print("\n🔍 VALIDATION: Testing pandas/numpy indexing fix...")

# Test the type coercion and indexing
print("\n📊 Testing type coercion and indexing...")

# Create test data
test_X = pd.DataFrame(np.random.randn(10, 5), columns=[f'feature_{i}' for i in range(5)])
test_y_np = np.random.randn(10)  # Start as numpy array
test_groups = ['group_A'] * 3 + ['group_B'] * 3 + ['group_C'] * 4

print(f"Initial types:")
print(f"  test_X type: {type(test_X)}")
print(f"  test_y_np type: {type(test_y_np)}")
print(f"  test_groups type: {type(test_groups)}")

# Apply the type coercion fix
if not isinstance(test_X, pd.DataFrame):
    test_X = pd.DataFrame(test_X)

if isinstance(test_y_np, np.ndarray):
    test_y = pd.Series(test_y_np, index=test_X.index, name='test_target')
else:
    test_y = test_y_np.reset_index(drop=True)
test_X = test_X.reset_index(drop=True)

print(f"\nAfter type coercion:")
print(f"  test_X type: {type(test_X)}")
print(f"  test_y type: {type(test_y)}")
print(f"  test_X has iloc: {hasattr(test_X, 'iloc')}")
print(f"  test_y has iloc: {hasattr(test_y, 'iloc')}")

# Test indexing
print(f"\nTesting indexing:")
print(f"  test_X.iloc[0:3] shape: {test_X.iloc[0:3].shape}")
print(f"  test_y.iloc[0:3] shape: {test_y.iloc[0:3].shape}")

# Test GroupKFold with proper indexing
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=3)
for fold, (tr, va) in enumerate(gkf.split(test_X, test_y, groups=test_groups), 1):
    print(f"  Fold {fold}: {len(tr)} train, {len(va)} val")
    
    # Test the indexing that was causing the error
    X_tr_raw, X_va_raw = test_X.iloc[tr].copy(), test_X.iloc[va].copy()
    y_tr, y_va = test_y.iloc[tr], test_y.iloc[va]  # This should work now!
    
    print(f"    X_tr_raw shape: {X_tr_raw.shape}")
    print(f"    y_tr shape: {y_tr.shape}")
    print(f"    X_va_raw shape: {X_va_raw.shape}")
    print(f"    y_va shape: {y_va.shape}")

print(f"\n✅ PANDAS/NUMPY INDEXING FIX VERIFIED!")
print(f"✅ Type coercion ensures proper pandas types")
print(f"✅ .iloc indexing works correctly on pandas objects")
print(f"✅ GroupKFold indexing is now consistent")
print(f"✅ No more AttributeError: 'numpy.ndarray' object has no attribute 'iloc'")


In [ ]:
# =============================================================================
# PROPER GROUPKFOLD USAGE VALIDATION
# =============================================================================

print("\n🔍 VALIDATION: Testing proper GroupKFold usage...")

# Test with proper training data (not test data)
print("\n📊 Testing GroupKFold on training data...")

# Create realistic training data with multiple groups
train_groups = ['group_A'] * 10 + ['group_B'] * 8 + ['group_C'] * 6 + ['group_D'] * 4 + ['group_E'] * 2
train_X = pd.DataFrame(np.random.randn(len(train_groups), 5), columns=[f'feature_{i}' for i in range(5)])
train_y = pd.Series(np.random.randn(len(train_groups)), name='target')

print(f"Training data: {len(train_X)} samples, {len(set(train_groups))} unique groups")

# Test proper GroupKFold usage
from sklearn.model_selection import GroupKFold

n_groups = len(np.unique(train_groups))
n_splits = min(5, n_groups)  # must be <= unique groups

print(f"Using {n_splits} folds for {n_groups} unique groups")

if n_splits < 2:
    print(f"⚠️ Not enough groups for CV: {n_groups} groups, need at least 2")
else:
    gkf = GroupKFold(n_splits=n_splits)
    
    for fold, (tr, va) in enumerate(gkf.split(train_X, train_y, groups=train_groups), 1):
        train_groups_fold = set([train_groups[i] for i in tr])
        val_groups_fold = set([train_groups[i] for i in va])
        
        print(f"  Fold {fold}: {len(tr)} train, {len(va)} val")
        print(f"    Train groups: {sorted(train_groups_fold)}")
        print(f"    Val groups: {sorted(val_groups_fold)}")
        print(f"    Overlap: {train_groups_fold & val_groups_fold}")
        
        # Verify no group leakage
        assert train_groups_fold.isdisjoint(val_groups_fold), f"Group leakage in fold {fold}!"

print(f"\n✅ PROPER GROUPKFOLD USAGE VERIFIED!")
print(f"✅ GroupKFold only used on training data")
print(f"✅ Proper group count validation prevents ValueError")
print(f"✅ No group leakage across folds")
print(f"✅ Test data handled separately (no CV on test)")

print(f"\n🎯 KEY PRINCIPLES:")
print(f"• GroupKFold is ONLY for training data cross-validation")
print(f"• Test data is used once for final predictions")
print(f"• n_splits must be ≤ number of unique groups")
print(f"• Always validate group counts before splitting")


In [ ]:
# =============================================================================
# PROPER TRAINING GROUPKFOLD VALIDATION
# =============================================================================

print("\n🔍 VALIDATION: Testing proper GroupKFold on TRAINING data...")

# Create realistic training data with multiple groups
train_groups = ['group_A'] * 10 + ['group_B'] * 8 + ['group_C'] * 6 + ['group_D'] * 4 + ['group_E'] * 2
train_X = pd.DataFrame(np.random.randn(len(train_groups), 5), columns=[f'feature_{i}' for i in range(5)])
train_y = pd.Series(np.random.randn(len(train_groups)), name='target')

print(f"Training data: {len(train_X)} samples, {len(set(train_groups))} unique groups")

# Test proper GroupKFold usage with safe guard
from sklearn.model_selection import GroupKFold
import numpy as np

groups_train = np.asarray(train_groups)  # canonical SMILES / polymer_id (TRAIN)
u = np.unique(groups_train).size
n_splits = min(5, u)  # must be ≤ unique groups

print(f"Unique training groups: {u}")
print(f"Using {n_splits} folds for {u} unique groups")

if n_splits < 2:
    print(f"⚠️ Not enough groups for CV: {u} groups, need at least 2")
    print(f"ℹ️ Would use single model or GroupShuffleSplit instead")
else:
    gkf = GroupKFold(n_splits=n_splits)
    
    for fold, (tr, va) in enumerate(gkf.split(train_X, train_y, groups=groups_train), 1):
        train_groups_fold = set([train_groups[i] for i in tr])
        val_groups_fold = set([train_groups[i] for i in va])
        
        print(f"  Fold {fold}: {len(tr)} train, {len(va)} val")
        print(f"    Train groups: {sorted(train_groups_fold)}")
        print(f"    Val groups: {sorted(val_groups_fold)}")
        print(f"    Overlap: {train_groups_fold & val_groups_fold}")
        
        # Verify no group leakage
        assert train_groups_fold.isdisjoint(val_groups_fold), f"Group leakage in fold {fold}!"

print(f"\n✅ PROPER TRAINING GROUPKFOLD VALIDATED!")
print(f"✅ GroupKFold only used on training data")
print(f"✅ Proper group count validation prevents ValueError")
print(f"✅ No group leakage across folds")
print(f"✅ Test data handled separately (no CV on test)")

# Test GroupShuffleSplit for cases with very few groups
print(f"\n🔧 TESTING GroupShuffleSplit for few groups...")
from sklearn.model_selection import GroupShuffleSplit

# Create data with only 2 groups
few_groups = ['group_A'] * 10 + ['group_B'] * 10
few_X = pd.DataFrame(np.random.randn(20, 5), columns=[f'feature_{i}' for i in range(5)])
few_y = pd.Series(np.random.randn(20), name='target')

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr, va = next(gss.split(few_X, few_y, groups=few_groups))

print(f"GroupShuffleSplit: {len(tr)} train, {len(va)} val")
print(f"✅ GroupShuffleSplit works for few groups")


In [ ]:
# =============================================================================
# GROUP GENERATION DIAGNOSTICS
# =============================================================================

print("\n🔍 DIAGNOSTICS: Checking for common group generation issues...")

def diagnose_groups(groups, name="groups"):
    """Diagnose common issues with group generation"""
    print(f"\n📊 Diagnosing {name}:")
    
    groups_array = np.asarray(groups)
    u = np.unique(groups_array).size
    print(f"  Unique groups: {u}")
    print(f"  Total samples: {len(groups_array)}")
    print(f"  Data type: {groups_array.dtype}")
    
    # Check for common issues
    vals, counts = np.unique(groups_array, return_counts=True)
    print(f"  Top 5 groups: {list(zip(vals[:5], counts[:5]))}")
    
    # Check for canonicalization collapse
    if u == 1:
        print(f"  ⚠️ WARNING: All groups are identical! Check canonicalization.")
    elif u < 5:
        print(f"  ⚠️ WARNING: Very few unique groups ({u}). Check canonicalization.")
    
    # Check for constant/boolean groups
    if groups_array.dtype == bool:
        print(f"  ⚠️ WARNING: Groups are boolean! Check group generation.")
    
    # Check for parent index issues
    if hasattr(groups, '__len__') and len(groups) > 0:
        if isinstance(groups[0], str) and 'INVALID' in str(groups[0]):
            print(f"  ⚠️ WARNING: Groups contain 'INVALID' - check canonicalization fallback.")
    
    return u

# Test the diagnostic function
print("Testing group diagnostics...")

# Test with good groups
good_groups = ['group_A', 'group_B', 'group_C', 'group_A', 'group_B']
u_good = diagnose_groups(good_groups, "good groups")

# Test with collapsed groups
collapsed_groups = ['INVALID', 'INVALID', 'INVALID', 'INVALID', 'INVALID']
u_collapsed = diagnose_groups(collapsed_groups, "collapsed groups")

# Test with few groups
few_groups = ['group_A', 'group_B', 'group_A', 'group_B', 'group_A']
u_few = diagnose_groups(few_groups, "few groups")

print(f"\n✅ GROUP DIAGNOSTICS COMPLETE!")
print(f"✅ Good groups: {u_good} unique")
print(f"✅ Collapsed groups: {u_collapsed} unique (should be 1)")
print(f"✅ Few groups: {u_few} unique")

print(f"\n🎯 COMMON FIXES:")
print(f"• If all groups are identical: Fix canonicalization fallback")
print(f"• If very few groups: Check parent index mapping")
print(f"• If boolean groups: Fix group generation logic")
print(f"• If 'INVALID' groups: Use unique fallback per row")


In [ ]:
# =============================================================================
# SAFE FEATURIZATION PATTERN VALIDATION
# =============================================================================

print("\n🔍 VALIDATION: Testing safe featurization pattern...")

# Test with realistic data including invalid SMILES
test_smiles = ['CCO', 'INVALID_SMILES', 'CCN', 'ANOTHER_INVALID', 'CCO', 'C1=CC=CC=C1']
test_labels = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

print(f"Test SMILES: {test_smiles}")
print(f"Test labels: {test_labels}")

# Test safe canonicalization
print("\n🔧 Testing safe canonicalization...")
canon_test = canon_smiles_list(test_smiles)
print(f"Canonical SMILES: {canon_test}")

# Verify no re-parsing of placeholders
invalid_canon = [c for c in canon_test if c.startswith('INVALID_')]
print(f"Invalid canonical SMILES: {invalid_canon}")
print(f"✅ No re-parsing of placeholders")

# Test safe descriptor generation
print("\n🔧 Testing safe descriptor generation...")
test_desc_list = [rdkit_descriptors_or_none(s) for s in test_smiles]
print(f"Descriptor results: {[type(d).__name__ for d in test_desc_list]}")

# Test safe fingerprint generation
print("\n🔧 Testing safe fingerprint generation...")
test_fp_list = [fingerprints_or_none(s, n_bits=1024) for s in test_smiles]
print(f"Fingerprint results: {[type(f).__name__ if f is not None else 'None' for f in test_fp_list]}")

# Test training data handling (drop invalid rows)
print("\n🔧 Testing training data handling...")
invalid_train = [i for i, d in enumerate(test_desc_list) if d is None or test_fp_list[i] is None]
print(f"Invalid indices: {invalid_train}")

if invalid_train:
    keep = np.setdiff1d(np.arange(len(test_smiles)), invalid_train)
    train_labels = np.array(test_labels)[keep]
    train_smiles = [test_smiles[i] for i in keep]
    train_desc_list = [test_desc_list[i] for i in keep]
    train_fp_list = [test_fp_list[i] for i in keep]
    
    print(f"Dropped {len(invalid_train)} invalid rows")
    print(f"Clean training data: {len(train_smiles)} samples")
else:
    train_labels = np.array(test_labels)
    train_smiles = test_smiles
    train_desc_list = test_desc_list
    train_fp_list = test_fp_list
    print(f"No invalid rows to drop")

# Test test data handling (keep all rows, fill invalid)
print("\n🔧 Testing test data handling...")
# Establish descriptor columns from clean training data
desc_cols = sorted({k for d in train_desc_list for k in d.keys()})
print(f"Descriptor columns: {len(desc_cols)}")

# Build test descriptors with safe fallbacks
test_desc_safe = []
for i, d in enumerate(test_desc_list):
    if d is None:
        test_desc_safe.append({c: 0.0 for c in desc_cols})
    else:
        test_desc_safe.append({c: d.get(c, 0.0) for c in desc_cols})

# Build test fingerprints with safe fallbacks
test_fp_safe = []
for i, f in enumerate(test_fp_list):
    if f is None:
        test_fp_safe.append(np.zeros(1024 + 167, dtype=np.uint8))
    else:
        test_fp_safe.append(f)

# Test DataFrame creation (should not crash)
print("\n🔧 Testing DataFrame creation...")
test_X = pd.DataFrame(test_desc_safe)
test_fingerprints = np.vstack(test_fp_safe)

print(f"✅ Safe featurization pattern verified!")
print(f"   Training data: {len(train_smiles)} samples")
print(f"   Test data: {len(test_X)} samples")
print(f"   Descriptors: {test_X.shape}")
print(f"   Fingerprints: {test_fingerprints.shape}")
print(f"   No None values in test descriptors")
print(f"   All test rows preserved for predictions")

# Test group generation with safe canonicalization
print("\n🔧 Testing group generation...")
test_groups = canon_smiles_list(test_smiles)
print(f"Test groups: {test_groups}")
print(f"Unique groups: {len(np.unique(test_groups))}")

print(f"\n✅ SAFE FEATURIZATION PATTERN COMPLETE!")
print(f"✅ No RDKit parse errors from placeholders")
print(f"✅ No AttributeError from None descriptors")
print(f"✅ Training data: drop invalid rows")
print(f"✅ Test data: keep all rows, fill invalid with zeros")
print(f"✅ Proper group generation with unique fallbacks")
